## Dataset: KDDCup

In [2]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
import sklearn
import numpy as np
import torch
from SADL.time_series.time_series_utils import TimeseriesDataset
from SADL.time_series.algorithms import tsfedl

data, attack_types, classes = dataset.readKDDCup99Dataset()
data = sklearn.preprocessing.StandardScaler().fit_transform(data)

attack_types_dict = {}
cont=0
for att in attack_types:
    attack_types_dict[att]=cont
    cont+=1

for i in range(len(classes)):
    classes[i] = attack_types_dict[classes[i]]

classes[classes!=attack_types_dict["normal"]] = 1
classes[classes==attack_types_dict["normal"]] = 0
classes_test = classes[-500000:]

normal_train = np.where(classes[:-500000]==0)[0][:10000]
train_dataset = TimeseriesDataset(torch.from_numpy(data[normal_train]).double(), seq_len=4)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 3, shuffle = False)




2024-02-29 08:34:23.275303: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 08:34:23.275484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 08:34:23.489223: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 08:34:23.950015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 08:34:26.010020: W tensorflow/compiler/tf2

In [1]:
import pickle as pkl
with open('test.pkl','wb') as f:
    pkl.dump(data[normal_train], f)

NameError: name 'data' is not defined

## Load KDDCup Dataset

In [26]:
import pickle as pkl
with open('test.pkl','rb') as f:
    train_data = pkl.load(f)


EOFError: Ran out of input

In [2]:
print(train_data.shape)

(10000, 126)


## MODEL OhShuLi

In [17]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

train_dataset_fuera = TimeSeriesDatasetV2(torch.from_numpy(train_data).double(), window_size=4, forecast_size=1,permute_size = (1,0))
train_loader_fuera = torch.utils.data.DataLoader(train_dataset_fuera, batch_size = 32, shuffle = True)



class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   
kwargs = {"algorithm_": "ohshulih", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=20, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model1 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model1.model = model1.model.double()

#print(model1.get_params())
print(model1.model)

#model1.fit(train_loader)

{'label_parser': True, 'algorithm_': 'OhShuLih', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
OhShuLih(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=20, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 3, kernel_size=(20,), stride=(1,), padding=(19,), bias=False)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(3, 6, kernel_size=(10,), stride=(1,), padding=(9,), bias=False)
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(6, 6, kernel_size=(5,), stride=(1,), padding=(4,), bias=False)
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1,

In [4]:
model1.fit(train_loader_fuera)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | loss         | MSELoss    | 0     
1 | classifier   | mia        | 7.5 K 
2 | convolutions | Sequential | 7.9 K 
3 | lstm         | LSTM       | 2.2 K 
--------------------------------------------
17.6 K    Trainable params
0         Non-trainable params
17.6 K    Total params
0.071     Total estimated model params size (MB)


Training: |                                                           | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([12, 1, 126])) that is different to the input size (torch.Size([12, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [5]:
scores = model1.decision_function(train_loader_fuera)

Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(27, 126)
Y Shape: (27, 126)
y_pred shape:(2

In [33]:
print(len(scores))
print(27 *train_loader_fuera.__len__())

9996
10017


## Modelo: LiOhShu


In [15]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_2 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_2 = torch.utils.data.DataLoader(train_dataset_2, batch_size = 1, shuffle = False)

kwargs = {"algorithm_": "liohshu", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=10, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_2)

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | loss         | MSELoss    | 0     
1 | classifier   | mia        | 7.0 K 
2 | convolutions | Sequential | 8.5 K 
3 | lstm         | LSTM       | 720   

{'label_parser': True, 'algorithm_': 'LihOhShu', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
LihOhShu(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=10, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 3, kernel_size=(20,), stride=(1,), bias=False)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(3, 6, kernel_size=(10,), stride=(1,), bias=False)
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(6, 6, kernel_size=(5,), stride=(1,), bias=False)
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv1d(6, 6, ker

Training: |                                                                                           | 0/? [0…

TSFEDLerror fit():  Given groups=1, weight of size [3, 126, 20], expected input[1, 400, 126] to have 126 channels, but got 400 channels instead
For further reference please see: https://s-tsfe-dl.readthedocs.io/en/latest/index.html


RuntimeError: Given groups=1, weight of size [3, 126, 20], expected input[1, 400, 126] to have 126 channels, but got 400 channels instead

In [43]:
scores = model2.decision_function(train_loader_2)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2684118/2808341109.py", line 1, in <module>
    scores = model2.decision_function(train_loader_2)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/homeGPU/mbautista/S-ADL/SADL/time_series/algorithms/tsfedl.py", line 71, in decision_function
Exception: TSFEDLerror decision_function(): y_pred.shape differs from y shape.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2144, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/si

## Modelo: Yibogao

In [8]:
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import torch
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeseriesDataset
import sklearn
import numpy as np
from SADL.time_series.algorithms import tsfedl


class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   

train_dataset_3 = TimeSeriesDatasetV2(torch.from_numpy(train_data).double(), window_size=400, forecast_size=1)
train_loader_3 = torch.utils.data.DataLoader(train_dataset_3, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "yibogao", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=1, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model3 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model3.model = model3.model.double()
print(model3.model)
model3.fit(train_loader_3)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | classifier | mia        | 6.5 K 
1 | module     | Sequential | 1.4 M 
2 | loss       | MSELoss    | 0     
------------------------------------------
1.4 M     Trainable params
0         Non-trainable params
1.4 M     Total params
5.762     Total estimated model params size (MB)


{'label_parser': True, 'algorithm_': 'YiboGao', 'pytorch_params_': {}, 'in_features': 126, 'loss': <function en_loss at 0x7f3a29d31bc0>, 'metrics': None, 'optimizer': None, 'top_module': None}
YiboGao(
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=1, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (module): Sequential(
    (0): RTABlock(
      (conv1): ConvBlockYiboGao(
        (module): Sequential(
          (0): Conv1d(126, 16, kernel_size=(32,), stride=(1,), padding=same)
          (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
        )
      )
      (conv2): ConvBlockYiboGao(
        (module): Sequential(
          (0): Conv1d(16, 16, kernel_size=(32,), stride=(1,), padding=same)
          (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       

Training: |                                                           | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [9]:
scores = model3.decision_function(train_loader_3)
len(scores)

9600

In [23]:
scores[:5]

array([4.56596286, 4.56822094, 4.57054428, 4.57313846, 4.57555748])

## Modelo YaoQihang

In [2]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_4 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_4 = torch.utils.data.DataLoader(train_dataset_4, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "yaoqihang", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=32, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_4)

2024-03-01 10:39:03.393743: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 10:39:03.393869: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 10:39:03.395567: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 10:39:03.420608: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 10:39:08.459160: W tensorflow/compiler/tf2

{'label_parser': True, 'algorithm_': 'YaoQihang', 'pytorch_params_': {}, 'in_features': 126, 'loss': None, 'metrics': None, 'optimizer': None, 'top_module': None}
YaoQihang(
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=32, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Sequential(
      (0): Conv1d(126, 64, kernel_size=(3,), stride=(1,), padding=same)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=same)
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
    (3): Sequential(
      (0): Conv1d(64, 128, kernel_si

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                                                           | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [6]:
scores = model2.decision_function(train_loader_4)

In [8]:
print(32 * train_loader_4.__len__())
print(len(scores))

9600
9600


## MODEL: HtetMyetLynn

In [9]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "htetmyetlynn", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=80, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...


{'label_parser': True, 'algorithm_': 'HtetMyetLynn', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None, 'use_rnn': None}
HtetMyetLynn(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=80, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convLayers): Sequential(
    (0): Conv1d(126, 30, kernel_size=(5,), stride=(1,), padding=same)
    (1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv1d(30, 30, kernel_size=(2,), stride=(1,), padding=same)
    (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv1d(30, 60, kernel_size=(5,), stride=(1,), padding=same)
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(60, 60, kernel_size=(2,),

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | loss       | MSELoss    | 0     
1 | classifier | mia        | 10.5 K
2 | convLayers | Sequential | 37.1 K
3 | rnn        | GRU        | 24.5 K
------------------------------------------
72.0 K    Trainable params
0         Non-trainable params
72.0 K    Total params
0.288     Total estimated model params size (MB)


Training: |                                                           | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1040.)
  return F.conv1d(input, weight, bias, self.stride,
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [11]:
scores = model2.decision_function(train_loader_5)

## Model: YildirimOzal - pendiente

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=126, forecast_size=1, permute_size = (1,0))
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "yildirimozal", "input_shape":(126,126), "loss": torch.nn.MSELoss(),"top_module": mia(in_features=32, out_features=126), "max_epochs": 1, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

{'label_parser': True, 'algorithm_': 'YildirimOzal', 'pytorch_params_': {}, 'input_shape': None, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None, 'train_autoencoder': True}
YildirimOzal(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=32, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (encoder): Sequential(
    (0): Conv1d(126, 16, kernel_size=(5,), stride=(1,), padding=same)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(16, 64, kernel_size=(5,), stride=(1,), padding=same)
    (4): ReLU()
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv1d(64, 32, kernel_size=(3,), stride=(1,), padding=sa

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                                                                                           | 0/? [0…

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 126, 200])) that is different to the input size (torch.Size([32, 1, 200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8, 126, 200])) that is different to the input size (torch.Size([8, 1, 200])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

(32, 200)
(32, 126)


Exception: TSFEDLerror decision_function(): y_pred.shape differs from y shape.

## MODEL: CaiWenjuan

In [5]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "caiwenjuan", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=67, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | loss         | MSELoss    | 0     
1 | classifier   | mia        | 9.8 K 
2 | conv1        | Conv1d     | 1.0 K 
3 | conv2        | Conv1d     | 6.1 K 
4 | conv3        | Conv1d     | 15.1 K
5 | dense_module | Sequential | 63.9 K
--------------------------------------------
95.9 K    Trainable params
0         Non-trainable params
95.9 K    Total params
0.384     Total estimated model params size (MB)


{'label_parser': True, 'algorithm_': 'CaiWenjuan', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'reduction_ratio': None, 'top_module': None}
CaiWenjuan(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=67, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (conv1): Conv1d(126, 8, kernel_size=(1,), stride=(1,), padding=same)
  (conv2): Conv1d(126, 16, kernel_size=(3,), stride=(1,), padding=same)
  (conv3): Conv1d(126, 24, kernel_size=(5,), stride=(1,), padding=same)
  (dense_module): Sequential(
    (0): SqueezeAndExcitationModule(
      (fully_connected): Sequential(
        (0): Linear(in_features=48, out_features=32, bias=True)
        (1): ReLU()
        (2): Linear(in_features=32, out_features=48, bias=True)
        (3): Sigmoid()
      )
    )
    (1): DenseNetDens

Training: |        | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [6]:
scores = model2.decision_function(train_loader_5)

(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)


## Model: ZhangJin

In [9]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "zhangjin", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=24, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type       | Params
--------------------------------------------------
0 | loss               | MSELoss    | 0     
1 | classifier         | mia        | 7.7 K 
2 | convolutions       | ModuleList | 1.8 M 
3 | spatial_attention  | ModuleList | 218 K 
4 | temporal_attention | ModuleList | 75    
5 | gru                | GRU        | 19.4 K
--------------------------------------------------
2.0 M     Trainable params
0         Non-trainable para

{'label_parser': True, 'algorithm_': 'ZhangJin', 'pytorch_params_': {}, 'decrease_ratio': 2, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
ZhangJin(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=24, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): ModuleList(
    (0): Sequential(
      (0): Sequential(
        (0): Conv1d(126, 64, kernel_size=(3,), stride=(1,), padding=same)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=same)
        (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (2): MaxPool1d(kernel_size=3, stride=3, p

Training: |        | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [10]:
scores = model2.decision_function(train_loader_5)

(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)
(32, 126)


## Model: KongZhengmin

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "kongzhengmin", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=64, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

2024-03-04 09:28:08.459166: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 09:28:08.459205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 09:28:08.460458: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 09:28:08.467173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 09:28:09.587839: W tensorflow/compiler/tf2

{'label_parser': True, 'algorithm_': 'KongZhengmin', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
KongZhengmin(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=64, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolution): Sequential(
    (0): Conv1d(126, 32, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(32, 64, num_layers=2, batch_first=True)
)


/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                         | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

## Model: WeiXiaoyan

In [4]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "weixiaoyan", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=512, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type        | Params
---------------------------------------------
0 | loss         | MSELoss     | 0     
1 | classifier   | mia         | 32.1 K
2 | convolutions | Sequential  | 545 K 
3 | lstm1        | LSTM        | 2.1 M 
4 | batchNorm    | BatchNorm1d | 1.0 K 
5 | lstm2        | LSTM        | 2.1 M 
---------------------------------------------
4.8 M     Trainable params
0         Non-trainable params
4.8 M     Total params
19.124    Total estimated model params size (MB)


{'label_parser': True, 'algorithm_': 'WeiXiaoyan', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
WeiXiaoyan(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=512, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 32, kernel_size=(5,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv1d(32, 64, kernel_size=(3,), stride=(1,))
    (5): LeakyReLU(negative_slope=0.01)
    (6): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

Training: |                         | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


In [5]:
scores = model2.decision_function(train_loader_5)

## Model: GaoJunLi

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "gaojunli", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=64, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

2024-03-04 09:34:11.570669: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 09:34:11.570712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 09:34:11.571969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 09:34:11.578702: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 09:34:12.760920: W tensorflow/compiler/tf2

{'label_parser': True, 'algorithm_': 'GaoJunLi', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': <class 'torch.optim.adam.Adam'>, 'optimizer': None, 'top_module': None}
GaoJunLi(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=64, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (lstm): LSTM(126, 64, dropout=0.3)
)


/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

## Model: KhanZulfiqar

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "khanzulfiqar", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=10, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

2024-03-04 09:59:47.826808: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 09:59:47.826850: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 09:59:47.828112: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 09:59:47.835200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 09:59:48.993998: W tensorflow/compiler/tf2

{'label_parser': True, 'algorithm_': 'KhanZulfiqar', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
KhanZulfiqar(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=10, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 3, kernel_size=(20,), stride=(1,), bias=False)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(3, 6, kernel_size=(10,), stride=(1,), bias=False)
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(6, 6, kernel_size=(5,), stride=(1,), bias=False)
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv1d(6

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

## Model: ZhengZhenyu

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "zhengzhenyu", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=256, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

2024-03-04 10:08:17.355424: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 10:08:17.355472: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 10:08:17.356738: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 10:08:17.363769: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 10:08:18.532052: W tensorflow/compiler/tf2

{'label_parser': True, 'algorithm_': 'ZhengZhenyu', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
ZhengZhenyu(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=256, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 64, kernel_size=(3,), stride=(1,), padding=same)
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=same)
    (4): ELU(alpha=1.0)
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same)
    (7): ELU(alpha=1.0)
    (8): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True,

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

## Model: WangKejun

In [4]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "wangkejun", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=256, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

{'label_parser': True, 'algorithm_': 'WangKejun', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
WangKejun(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=256, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 64, kernel_size=(3,), stride=(1,), padding=same)
    (1): ELU(alpha=1.0)
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=same)
    (4): ELU(alpha=1.0)
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=same)
    (7): ELU(alpha=1.0)
    (8): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, tra

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [5]:
scores = model2.decision_function(train_loader_5)

## Model: ChenChen

In [5]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=1000, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "chenchen", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=64, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | loss         | MSELoss    | 0     
1 | classifier   | mia        | 9.7 K 
2 | convolutions | Sequential | 766 K 
3 | lstm1        | LSTM       | 6.1 K 
4 | lstm2        | LSTM       | 25.1 K
--------------------------------------------
807 K     Trainable params
0         Non-trainable params
807 K     Total params
3.229     Total estimated model params size (MB)


{'label_parser': True, 'algorithm_': 'ChenChen', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
ChenChen(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=64, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 251, kernel_size=(5,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(251, 150, kernel_size=(5,), stride=(1,), padding=valid)
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv1d(150, 100, kernel_size=(10,), stride=(1,), padding=valid)
    (7): ReLU()
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9)

Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8, 1, 126])) that is different to the input size (torch.Size([8, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [6]:
scores = model2.decision_function(train_loader_5)

## Model: KimTaeYoung

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=1000, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "kimtaeyoung", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=64, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

{'label_parser': True, 'algorithm_': 'KimTaeYoung', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
KimTaeYoung(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=64, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 64, kernel_size=(2,), stride=(1,), padding=valid)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(64, 64, kernel_size=(2,), stride=(1,), padding=valid)
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(64, 64, batch_first=True)
)


/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([8, 1, 126])) that is different to the input size (torch.Size([8, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

## Model: GenMinxing pendiente

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl
"""
data, attack_types, classes = dataset.readKDDCup99Dataset()
data = sklearn.preprocessing.StandardScaler().fit_transform(data)

attack_types_dict = {}
cont=0
for att in attack_types:
    attack_types_dict[att]=cont
    cont+=1

for i in range(len(classes)):
    classes[i] = attack_types_dict[classes[i]]

classes[classes!=attack_types_dict["normal"]] = 1
classes[classes==attack_types_dict["normal"]] = 0
classes_test = classes[-500000:]

normal_train = np.where(classes[:-500000]==0)[0][:10000]
"""

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(train_data).double(), window_size=100, forecast_size=1, permute_size = (0,1))
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = True)

kwargs = {"algorithm_": "genminxing", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=80, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type    | Params
---------------------------------------
0 | loss       | MSELoss | 0     
1 | classifier | mia     | 10.5 K
2 | lstm       | LSTM    | 53.8 K
---------------------------------------
64.2 K    Trainable params
0         Non-trainable params
64.2 K    Total params
0.257     Total estimated model params size (MB)


{'label_parser': True, 'algorithm_': 'GenMinxing', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
GenMinxing(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=80, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (lstm): LSTM(126, 40, batch_first=True, bidirectional=True)
)


Training: |               | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([12, 1, 126])) that is different to the input size (torch.Size([12, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

In [4]:
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data[normal_train]).double(), window_size=100, forecast_size=1, permute_size = (1,0))
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = True)
kwargs = {"algorithm_": "genminxing", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=80, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

TypeError: TimeSeriesDatasetV2.__init__() got an unexpected keyword argument 'permute_size'

In [ ]:
scores = model2.decision_function(train_loader_5)

## Model: FuJiangmeng

In [2]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "fujiangmeng", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=256, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

2024-03-04 11:07:23.652206: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-04 11:07:23.652250: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-04 11:07:23.653497: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-04 11:07:23.660724: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-04 11:07:24.720684: W tensorflow/compiler/tf2

{'label_parser': True, 'algorithm_': 'FuJiangmeng', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
FuJiangmeng(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=256, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 32, kernel_size=(1,), stride=(1,), padding=same)
    (1): Tanh()
    (2): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(32, 256, batch_first=True, dropout=0.3)
)


/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [3]:
scores = model2.decision_function(train_loader_5)

## Model: ShiHaotian

In [5]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "shihaotian", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=32, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type       | Params
---------------------------------------------
0 | loss          | MSELoss    | 0     
1 | classifier    | mia        | 8.1 K 
2 | convolutions1 | Sequential | 52.4 K
3 | convolutions2 | Sequential | 

{'label_parser': True, 'algorithm_': 'ShiHaotian', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
ShiHaotian(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=32, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions1): Sequential(
    (0): Conv1d(126, 32, kernel_size=(13,), stride=(2,), padding=valid)
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convolutions2): Sequential(
    (0): Conv1d(126, 32, kernel_size=(13,), stride=(1,), padding=valid)
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (convolutions3): Sequential(
    (0): Conv1d(126, 32, kernel_size=(13,), stride=(2,), 

Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

## Model: HuangMeiLing

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=252, forecast_size=1, permute_size=(1,0))
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "huangmeiling", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=19, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | loss         | MSELoss    | 0     
1 | classifier   | mia        | 7.4 K 
2 | convolutions | Sequential | 177 K 
--------------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


{'label_parser': True, 'algorithm_': 'HuangMeiLing', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
HuangMeiLing(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=19, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): ConstantPad1d(padding=(1, 1), value=0)
    (1): Conv1d(126, 48, kernel_size=(15,), stride=(6,), padding=valid)
    (2): ReLU()
    (3): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (4): ConstantPad1d(padding=(3, 3), value=0)
    (5): Conv1d(48, 256, kernel_size=(7,), stride=(2,), padding=valid)
    (6): ReLU()
    (7): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
)


Training: |                                                                                           | 0/? [0…

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([20, 1, 126])) that is different to the input size (torch.Size([20, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

## Model: HongTan

In [2]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1, permute_size=(1,0))
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "hongtan", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=4, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

2024-03-05 11:38:01.408173: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 11:38:01.408205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 11:38:01.409427: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 11:38:01.416723: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-05 11:38:02.602529: W tensorflow/compiler/tf2

{'label_parser': True, 'algorithm_': 'HongTan', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
HongTan(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=4, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 40, kernel_size=(5,), stride=(1,), bias=False)
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv1d(40, 32, kernel_size=(3,), stride=(1,), bias=False)
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm1): LSTM(32, 32, batch_first=True)
  (lstm2): LSTM(32, 16, batch_first=True)
  (lstm3): LSTM(16, 4, batch_first=True)
)


/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You d

Training: |                                                                                           | 0/? [0…

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [3]:
scores = model2.decision_function(train_loader_5)

## Model: SharPar

In [3]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=400, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "sharpar", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=16, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type       | Params
--------------------------------------------
0 | loss         | MSELoss    | 0     
1 | classifier   | mia        | 7.3 K 
2 | convolutions | Sequential | 40.4 K
3 | lstm1        | LSTM       | 12.5 K
4 | lstm2        | LSTM       | 3.2 K 
5 | fcc_module   | Sequential | 544   
--------------------------------------------
63.9 K    Trainable params
0         Non-trainable params
63.9 K    Total params
0.256     Total estimated model params size (MB)


{'label_parser': True, 'algorithm_': 'SharPar', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
SharPar(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=16, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (convolutions): Sequential(
    (0): Conv1d(126, 64, kernel_size=(5,), stride=(1,), padding=same)
    (1): ReLU()
  )
  (lstm1): LSTM(64, 32, batch_first=True)
  (lstm2): LSTM(32, 16, batch_first=True)
  (fcc_module): Sequential(
    (0): Linear(in_features=16, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
    (3): ReLU()
  )
)


Training: |                | 0/? [00:00<?, ?it/s]

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([32, 1, 126])) that is different to the input size (torch.Size([32, 126])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
`Trainer.fit` stopped: `max_epochs=1` reached.


In [4]:
scores = model2.decision_function(train_loader_5)

## Model: DaiXiLi

In [17]:
import sys
sys.path.append("/mnt/homeGPU/mbautista/S-ADL")
import SADL.time_series.time_series_datasets as dataset
from SADL.time_series.time_series_utils import TimeSeriesDatasetV2
import sklearn
import numpy as np
import torch
from SADL.time_series.algorithms import tsfedl

class mia(torch.nn.Module):
    def __init__(self, in_features=103, out_features=103, npred=1):
        super(mia, self).__init__()
        self.npred = npred
        self.model = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=in_features, out_features=50),
            torch.nn.ReLU(),
            torch.nn.Linear(in_features=50, out_features=npred*out_features)
        )

    def forward(self, x):
        out = self.model(x)
        if len(out.shape)>2:
            out = out[:, -1, :]
        if self.npred > 1:
            # Reshape to (batch_size, npred, out_features)
            out = out.reshape(out.shape[0], self.npred, -1)
        return out   


data = train_data
train_dataset_5 = TimeSeriesDatasetV2(torch.from_numpy(data).double(), window_size=1000, forecast_size=1)
train_loader_5 = torch.utils.data.DataLoader(train_dataset_5, batch_size = 32, shuffle = False)

kwargs = {"algorithm_": "daixili", "loss": torch.nn.MSELoss(),"top_module": mia(in_features=2048, out_features=126), "max_epochs": 1, "in_features":126, "label_parser": True}
model2 = tsfedl.TsfedlAnomalyDetection(**kwargs)
model2.model = model2.model.double()
print(model2.model)
model2.fit(train_loader_5)

/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/homeGPU/mbautista/s-adl-environment/lib/python3 ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/homeGPU/mbautista/s-adl-environment/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name       | Type      | Params
------------------------------------------
0  | loss       | MSELoss   | 0     
1  | classifier | mia       | 108 K 
2  | v2c1       | Conv1d    | 10.3 K
3  | v2mp1      | MaxPool1d | 0     
4  | v2d1  

{'label_parser': True, 'algorithm_': 'DaiXiLi', 'pytorch_params_': {}, 'in_features': 126, 'loss': CrossEntropyLoss(), 'metrics': None, 'optimizer': None, 'top_module': None}
DaiXiLi(
  (loss): MSELoss()
  (classifier): mia(
    (model): Sequential(
      (0): Dropout(p=0.2, inplace=False)
      (1): Linear(in_features=2048, out_features=50, bias=True)
      (2): ReLU()
      (3): Linear(in_features=50, out_features=126, bias=True)
    )
  )
  (v2c1): Conv1d(64, 32, kernel_size=(5,), stride=(1,), padding=same)
  (v2mp1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (v2d1): Linear(in_features=125, out_features=16, bias=True)
  (v3c1): Conv1d(126, 128, kernel_size=(5,), stride=(1,), padding=same)
  (v3mp1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (v3c2): Conv1d(128, 64, kernel_size=(5,), stride=(1,), padding=same)
  (v3mp2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (v3c3): Conv1d(64, 

Training: |        | 0/? [00:00<?, ?it/s]

TSFEDLerror fit():  mat1 and mat2 shapes cannot be multiplied (2048x100 and 250x16)
For further reference please see: https://s-tsfe-dl.readthedocs.io/en/latest/index.html


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2048x100 and 250x16)

In [18]:
scores = model2.decision_function(train_loader_5)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2048x100 and 250x16)